### Using OpenAI gpt-4o-mini model to generate social media posts for events

In [10]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [11]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [12]:
openai = OpenAI()

In [13]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt
    
# Generate a summary of content fetched by scraping the website

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
# Step 1: Create your prompts
WEBSITE_LINK = "https://www.eventbrite.ie/e/startup-mastery-leveraging-lean-startup-agile-and-design-thinking-tickets-920474252267?aff=ebdssbcategorybrowse&keep_tld=1"

system_prompt = "You are an assistant that analyzes the contents of an event \
and provides short summaries for a Twitter post, an instagram post and a facebook post.\
Ensure the summaries abide by the platform rules for each of the platforms."

website_summary = summarize(WEBSITE_LINK)
user_prompt = f"The events details are as follows: {website_summary}. Please summarize the above. Capture details like time and location, please capture them as well."

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

# Step 3: Call OpenAI

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

# Step 4: print the result

display(Markdown(response.choices[0].message.content))

**Twitter Post:**  
🚀 Join us online for #StartupMastery on Jan 7, 6-9 PM GMT! Explore Lean Startup, Agile, & Design Thinking methodologies. Gain practical skills, access resources, and earn a certificate! Tickets from €74.98. Don't miss out! 🎟️🌟

**Instagram Post:**  
🌟 Ready to boost your startup skills? Join us for **Startup Mastery**! 💡 On January 7 from 6 PM to 9 PM GMT, dive into Lean Startup, Agile, and Design Thinking with top-notch experts. Access recorded sessions, worksheets, and get certified! 🎟️ Tickets from €74.98. See you online! 🚀✨ #StartupMastery #LeanStartup #Agile #DesignThinking

**Facebook Post:**  
🗓️ Exciting opportunity for entrepreneurs and startup enthusiasts! Attend our **Startup Mastery** online workshop on January 7, from 6 PM to 9 PM GMT. Learn about Lean Startup, Agile, and Design Thinking methodologies to enhance your startup journey. Enjoy a transformative experience with insights on MVP development, rapid prototyping, and feedback loops. Plus, you'll get access to recorded sessions and can earn a certificate! Limited tickets available from €74.98. Organizers: Lean Agile Zone. Don’t miss out! 🚀 #StartupMastery #Entrepreneurship